### Explore the data

In [25]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from random import seed
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
print('nominatim imported')
print('Libraries imported.')

nominatim imported
Libraries imported.


### Define Foursquare Credentials and Version¶

In [26]:
CLIENT_ID = 'VBK130EFHFXH3R5ZXE5QBDBOOMIF5M0DY4MUDFXRO3ACL5F1' # your Foursquare ID
CLIENT_SECRET = 'G0BMGCDKIDMDUSSHVN23VI405YYDZEVR1ONDA1WEENRJELKA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VBK130EFHFXH3R5ZXE5QBDBOOMIF5M0DY4MUDFXRO3ACL5F1
CLIENT_SECRET:G0BMGCDKIDMDUSSHVN23VI405YYDZEVR1ONDA1WEENRJELKA


### We are located in Rome

In [27]:
address = 'Rome, Italy'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.894802 12.4853384


### Searching hotels around Rome

In [28]:
search_query = 'Hotel'
radius = 1300
print(search_query + ' .... OK!')

Hotel .... OK!


In [29]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VBK130EFHFXH3R5ZXE5QBDBOOMIF5M0DY4MUDFXRO3ACL5F1&client_secret=G0BMGCDKIDMDUSSHVN23VI405YYDZEVR1ONDA1WEENRJELKA&ll=41.894802,12.4853384&v=20180604&query=Hotel&radius=1300&limit=100'

### Results of hotels in Rome

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9271436a60715344885b59'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'primary': True,
      'shortName': 'Hotel'}],
    'hasPerk': False,
    'id': '4ba5f478f964a520182b39e3',
    'location': {'address': "Via Tor De' Conti, 25",
     'cc': 'IT',
     'city': 'Roma',
     'country': 'Italia',
     'distance': 225,
     'formattedAddress': ["Via Tor De' Conti, 25",
      '00184 Roma Lazio',
      'Italia'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.89384,
       'lng': 12.48773}],
     'lat': 41.89384,
     'lng': 12.48773,
     'postalCode': '00184',
     'state': 'Lazio'},
    'name': 'Hotel Forum',
    'referralId': 'v-1553101123'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     

In [31]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
print(dataframe.shape)
dataframe.head()

(50, 19)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,4ba5f478f964a520182b39e3,"Via Tor De' Conti, 25",IT,Roma,Italia,NaN,225,"[Via Tor De' Conti, 25, 00184 Roma Lazio, Italia]","[{'lng': 12.48773, 'lat': 41.89384, 'label': '...",41.893840,12.487730,NaN,00184,Lazio,Hotel Forum,v-1553101123,NaN
1,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,4adcdabff964a520735121e3,Via 4 Novembre 104,IT,Roma,Italia,NaN,233,"[Via 4 Novembre 104, 00187 Roma Lazio, Italia]","[{'lng': 12.484983726163296, 'lat': 41.8968868...",41.896887,12.484984,NaN,00187,Lazio,Hotel Pace Helvezia,v-1553101123,NaN
2,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,4bc889c3dc55eee1a389e8ac,Via dei Serpenti 109,IT,Roma,Italia,NaN,447,"[Via dei Serpenti 109, 00184 Roma Lazio, Italia]","[{'lng': 12.490096515873827, 'lat': 41.8967011...",41.896701,12.490097,NaN,00184,Lazio,Hotel Apollo,v-1553101123,NaN
3,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,4bcc607faeaaeee10bf53d6d,Via Genova 24,IT,Roma,Italia,NaN,794,"[Via Genova 24, Roma Lazio, Italia]","[{'lng': 12.492568476036832, 'lat': 41.8994855...",41.899486,12.492568,NaN,NaN,Lazio,Hotel Galatea,v-1553101123,NaN
4,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,4bc88aebdc55eee1808ae8ac,Via Urbana 3,IT,Roma,Italia,NaN,913,"[Via Urbana 3, 00184 Roma Lazio, Italia]","[{'lng': 12.495622349856664, 'lat': 41.8977699...",41.897770,12.495622,NaN,00184,Lazio,Hotel Raffaello,v-1553101123,NaN


### Creating a dataframe filtered

In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hotel Forum,Hotel,"Via Tor De' Conti, 25",IT,Roma,Italia,NaN,225,"[Via Tor De' Conti, 25, 00184 Roma Lazio, Italia]","[{'lng': 12.48773, 'lat': 41.89384, 'label': '...",41.893840,12.487730,NaN,00184,Lazio,4ba5f478f964a520182b39e3
1,Hotel Pace Helvezia,Hotel,Via 4 Novembre 104,IT,Roma,Italia,NaN,233,"[Via 4 Novembre 104, 00187 Roma Lazio, Italia]","[{'lng': 12.484983726163296, 'lat': 41.8968868...",41.896887,12.484984,NaN,00187,Lazio,4adcdabff964a520735121e3
2,Hotel Apollo,Hotel,Via dei Serpenti 109,IT,Roma,Italia,NaN,447,"[Via dei Serpenti 109, 00184 Roma Lazio, Italia]","[{'lng': 12.490096515873827, 'lat': 41.8967011...",41.896701,12.490097,NaN,00184,Lazio,4bc889c3dc55eee1a389e8ac
3,Hotel Galatea,Hotel,Via Genova 24,IT,Roma,Italia,NaN,794,"[Via Genova 24, Roma Lazio, Italia]","[{'lng': 12.492568476036832, 'lat': 41.8994855...",41.899486,12.492568,NaN,NaN,Lazio,4bcc607faeaaeee10bf53d6d
4,Hotel Raffaello,Hotel,Via Urbana 3,IT,Roma,Italia,NaN,913,"[Via Urbana 3, 00184 Roma Lazio, Italia]","[{'lng': 12.495622349856664, 'lat': 41.8977699...",41.897770,12.495622,NaN,00184,Lazio,4bc88aebdc55eee1808ae8ac
5,Hotel Hassler Roma,Hotel,Piazza Trinità dei Monti 6,IT,Roma,Italia,NaN,1245,"[Piazza Trinità dei Monti 6, 00187 Roma Lazio,...","[{'lng': 12.483731501780328, 'lat': 41.9059298...",41.905930,12.483732,NaN,00187,Lazio,4adcdabef964a5200c5121e3
6,Hotel Diana,Hotel,Via Principe Amedeo 4,IT,Roma,Italia,NaN,1193,"[Via Principe Amedeo 4, 00185 Roma Lazio, Italia]","[{'lng': 12.49740204661522, 'lat': 41.90065681...",41.900657,12.497402,NaN,00185,Lazio,4adcdabef964a5201c5121e3
7,Hotel Esposizione,Hotel,"Via Nazionale, 66",IT,Roma,Italia,NaN,716,"[Via Nazionale, 66, 00187 Roma Lazio, Italia]","[{'lng': 12.491481627216341, 'lat': 41.8993283...",41.899328,12.491482,NaN,00187,Lazio,4ce6810e8ef78cfa34cb8c9b
8,Hotel Fori Imperiali Cavalieri,Hotel,Via Frangipane 34,IT,Roma,Italia,NaN,387,"[Via Frangipane 34, 00184 Roma Lazio, Italia]","[{'lng': 12.4894635100329, 'lat': 41.893165110...",41.893165,12.489464,NaN,00184,Lazio,4c0c1a17a1b32d7f830f9cf0
9,Hotel Mediterraneo,Roof Deck,Via Cavour 15,IT,Roma,Italia,NaN,1288,"[Via Cavour 15, Roma Lazio, Italia]","[{'lng': 12.498987243459519, 'lat': 41.9003502...",41.900350,12.498987,NaN,NaN,Lazio,4bc8c75b762beee1cc853d38


## Metholodogy

In this project we will direct our efforts on detecting venues around the hotels. We will limit our analysis to area 500 meters around each hotel.

In first step we have collected the required data: location and type (category) of every venue within 500 meters around each hotel,we will use maps to visualize the hotels in Rome.

Second step in our analysis will be calculation and exploration of 'venues' across different areas of hootels-

In third and final step we will focus on most promising areas and within those create clusters of locations that meet the requirements established in discussion with stakeholders.

### Analysis

In [33]:
df=dataframe_filtered[['name','categories','lat','lng']]
print(df.shape)
df.head()

(50, 4)


,name,categories,lat,lng
0,Hotel Forum,Hotel,41.893840,12.487730
1,Hotel Pace Helvezia,Hotel,41.896887,12.484984
2,Hotel Apollo,Hotel,41.896701,12.490097
3,Hotel Galatea,Hotel,41.899486,12.492568
4,Hotel Raffaello,Hotel,41.897770,12.495622


### Define functions

The first function is to get the names of categories and the second is to get all the venues in a radius of 500 meters

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get all venues aroud each hotel

In [36]:
hotels_venues = getNearbyVenues(names=df['name'],
                                   latitudes=df['lat'],
                                   longitudes=df['lng']
                                  )
print(hotels_venues.shape)

Hotel Forum
Hotel Pace Helvezia
Hotel Apollo
Hotel Galatea
Hotel Raffaello
Hotel Hassler Roma
Hotel Diana
Hotel Esposizione
Hotel Fori Imperiali Cavalieri
Hotel Mediterraneo
iQ Hotel Roma
Hotel Giolli
The Fifteen Keys Hotel
Hotel Capo D'Africa Rome
Hotel Sonya
Hotel Ambasciatori Palace
Hotel Rex Rome
Welcome Piram Hotel Rome
939 Hotel (Hotel Marcus)
Grand Hotel Palace
Hotel Amalfi Rome
Hotel Sistina - Roma
Hotel Gladiatori Palazzo Manfredi
Hotel D'Este
Hotel Art
Hotel San Anselmo
Baglioni Hotel Regina
Hotel Cecil Rome
Downtown Accommodation Hotel
Hotel Campo De' Fiori
Raphaël Hotel
Hotel Concordia
Hotel Miami Rome
Edera Hotel
Hotel Concorde
Harry's Bar Trevi Hotel & Restaurant
La Griffe Hotel Rome
Hotel Orlanda
Hotel De Monti
Hotel Diocleziano
Hotel Arenula
Aleph Rome Hotel, Curio Collection by Hilton
Hotel Scalinata di Spagna
Hotel Domus Romana
Hotel Saturnia
B&B Trevi Hotel
Hotel Martini
B&B Hotel
Hotel Cosmopolita Rome
Hotel Alius
(4639, 7)


### Exploring the new dataframe

In [37]:
dele=hotels_venues[(hotels_venues['Venue Category'] == 'Hotel') | (hotels_venues['Venue Category'] == 'Hostel')].index
hotels_venues=hotels_venues.drop(dele,axis=0).reset_index(drop=True)
print(hotels_venues.shape)

(3728, 7)


### Counting all the venues around each hotel

In [38]:
hotels_venues.groupby('name').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
name,,,,,,
939 Hotel (Hotel Marcus),96,96,96,96,96,96
"Aleph Rome Hotel, Curio Collection by Hilton",66,66,66,66,66,66
B&B Hotel,81,81,81,81,81,81
B&B Trevi Hotel,94,94,94,94,94,94
Baglioni Hotel Regina,69,69,69,69,69,69
Downtown Accommodation Hotel,93,93,93,93,93,93
Edera Hotel,47,47,47,47,47,47
Grand Hotel Palace,69,69,69,69,69,69
Harry's Bar Trevi Hotel & Restaurant,95,95,95,95,95,95


In [39]:
print('There are {} uniques categories.'.format(len(hotels_venues['Venue Category'].unique())))

There are 133 uniques categories.


### Transform categorical variables to indicator variables

In [40]:
hotels_onehot = pd.get_dummies(hotels_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotels_onehot['name'] = hotels_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [hotels_onehot.columns[-1]] + list(hotels_onehot.columns[:-1])
hotels_onehot = hotels_onehot[fixed_columns]

print('shape of hotels_onehot:',hotels_onehot.shape)
hotels_onehot.head()

shape of hotels_onehot: (3728, 134)


,name,Abruzzo Restaurant,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Courthouse,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Embassy / Consulate,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lingerie Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Nightclub,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Piano Bar,Pie Shop,Pizza Place,Plaza,Pub,Public Art,Record Shop,Recording Studio,Resort,Restaurant,Road,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Szechuan Restaurant,Tea Room,Temple,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Here we can find the frequencies of each venue in each hotels

In [41]:
hotels_grouped = hotels_onehot.groupby('name').mean().reset_index()
print('shape of hotels_grouped:',hotels_grouped.shape)
hotels_grouped.head()

shape of hotels_grouped: (50, 134)


,name,Abruzzo Restaurant,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Courthouse,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Embassy / Consulate,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lingerie Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Nightclub,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Piano Bar,Pie Shop,Pizza Place,Plaza,Pub,Public Art,Record Shop,Recording Studio,Resort,Restaurant,Road,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Szechuan Restaurant,Tea Room,Temple,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,939 Hotel (Hotel Marcus),0.0,0.0,0.000000,0.0,0.000000,0.031250,0.0,0.0,0.010417,0.000000,0.000000,0.0,0.0,0.031250,0.000000,0.000000,0.041667,0.000000,0.0,0.000000,0.010417,0.0,0.0,0.020833,0.0,0.010417,0.010417,0.000000,0.020833,0.010417,0.010417,0.000000,0.010417,0.010417,0.0,0.0,0.0,0.0,0.000000,0.010417,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.031250,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.072917,0.000000,0.0,0.218750,0.010417,0.000000,0.010417,0.010417,0.0,0.0,0.010417,0.0,0.000000,0.0,0.000000,0.031250,0.000000,0.0,0.000000,0.010417,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.010417,0.041667,0.093750,0.000000,0.000000,0.0,0.0,0.0,0.031250,0.010417,0.010417,0.000000,0.031250,0.0,0.010417,0.0,0.0,0.010417,0.010417,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.031250,0.041667,0.0,0.0,0.010417,0.0,0.0,0.010417
1,"Aleph Rome Hotel, Curio Collection by Hilton",0.0,0.0,0.015152,0.0,0.000000,0.015152,0.0,0.0,0.000000,0.030303,0.060606,0.0,0.0,0.000000,0.015152,0.015152,0.015152,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.030303,0.0,0.015152,0.000000,0.000000,0.000000,0.045455,0.015152,0.000000,0.015152,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.015152,0.0,0.0,0.030303,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.272727,0.015152,0.015152,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.015152,0.0,0.030303,0.000000,0.015152,0.0,0.015152,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.015152,0.000000,0.045455,0.030303,0.000000,0.000000,0.0,0.0,0.0,0.045455,0.000000,0.015152,0.000000,0.000000,0.0,0.015152,0.0,0.0,0.000000,0.000000,0.015152,0.000000,0.0,0.0,0.0,0.0,0.015152,0.0,0.015152,0.000000,0.0,0.015152,0.060606,0.0,0.0,0.000000,0.0,0.0,0.000000
2,B&B Hotel,0.0,0.0,0.000000,0.0,0.000000,0.012346,0.0,0.0,0.012346,0.012346,0.037037,0.0,0.0,0.012346,0.000000,0.000000,0.000000,0.012346,0.0,0.000000,0.000000,0.0,0.0,0.037037,0.0,0.012346,0.000000,0.012346,0.000000,0.024691,0.037037,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.012346,0.000000,0.0,0.0,0.012346

### See the hotels with the frequencies

In [42]:
num_top_venues = 5

for hood in hotels_grouped['name']:
    print("----"+hood+"----")
    temp = hotels_grouped[hotels_grouped['name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----939 Hotel (Hotel Marcus)----
                venue  freq
0  Italian Restaurant  0.22
1               Plaza  0.09
2      Ice Cream Shop  0.07
3            Boutique  0.04
4         Pizza Place  0.04


----Aleph Rome Hotel, Curio Collection by Hilton----
                venue  freq
0  Italian Restaurant  0.27
1     Bed & Breakfast  0.06
2   Trattoria/Osteria  0.06
3         Pizza Place  0.05
4          Restaurant  0.05


----B&B Hotel----
                venue  freq
0  Italian Restaurant  0.25
1         Pizza Place  0.09
2      Ice Cream Shop  0.09
3               Plaza  0.05
4            Wine Bar  0.05


----B&B Trevi Hotel----
                venue  freq
0  Italian Restaurant  0.20
1      Ice Cream Shop  0.09
2               Plaza  0.07
3            Fountain  0.03
4                Café  0.03


----Baglioni Hotel Regina----
                venue  freq
0  Italian Restaurant  0.23
1   Trattoria/Osteria  0.06
2          Restaurant  0.06
3              Bistro  0.06
4         Pizza Place 

#### This function return the 'n' first venues

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Make a Dataframe of each hotels with their 10 top venues

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
names_venues_sorted = pd.DataFrame(columns=columns)
names_venues_sorted['name'] = hotels_grouped['name']

for ind in np.arange(hotels_grouped.shape[0]):
    names_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hotels_grouped.iloc[ind, :], num_top_venues)

names_venues_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,939 Hotel (Hotel Marcus),Italian Restaurant,Plaza,Ice Cream Shop,Pizza Place,Boutique,Trattoria/Osteria,Sandwich Place,Restaurant,Art Museum,Toy / Game Store
1,"Aleph Rome Hotel, Curio Collection by Hilton",Italian Restaurant,Bed & Breakfast,Trattoria/Osteria,Pizza Place,Restaurant,Cocktail Bar,Bar,Middle Eastern Restaurant,Fountain,Café
2,B&B Hotel,Italian Restaurant,Pizza Place,Ice Cream Shop,Plaza,Wine Bar,Café,Coffee Shop,Sandwich Place,Bed & Breakfast,Cocktail Bar
3,B&B Trevi Hotel,Italian Restaurant,Ice Cream Shop,Plaza,Art Museum,Monument / Landmark,Café,Bed & Breakfast,Sandwich Place,Restaurant,Toy / Game Store
4,Baglioni Hotel Regina,Italian Restaurant,Bistro,Restaurant,Trattoria/Osteria,Cocktail Bar,Pizza Place,Movie Theater,Café,Gym,Nightclub


### Starting the K-means segmentation

In [45]:
kclusters = 4
seed(27)
hotels_grouped_clustering = hotels_grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotels_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 2, 3, 3, 1, 3, 3, 0, 0], dtype=int32)

### Inserting cluster labels to the dataframe

In [46]:
# add clustering labels
names_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hotels_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hotels_merged = hotels_merged.join(names_venues_sorted.set_index('name'), on='name')

hotels_merged.head() # check the last columns!

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hotel Forum,Hotel,41.893840,12.487730,1,Italian Restaurant,Historic Site,Temple,History Museum,Wine Bar,Plaza,Pizza Place,Monument / Landmark,Sandwich Place,Ice Cream Shop
1,Hotel Pace Helvezia,Hotel,41.896887,12.484984,1,Italian Restaurant,Historic Site,Ice Cream Shop,History Museum,Plaza,Restaurant,Art Museum,Monument / Landmark,Pizza Place,Pub
2,Hotel Apollo,Hotel,41.896701,12.490097,2,Italian Restaurant,Pizza Place,Ice Cream Shop,Wine Bar,Coffee Shop,Bed & Breakfast,Sandwich Place,Historic Site,Indian Restaurant,Roman Restaurant
3,Hotel Galatea,Hotel,41.899486,12.492568,2,Italian Restaurant,Ice Cream Shop,Bed & Breakfast,Pizza Place,Wine Bar,Plaza,Cocktail Bar,Roman Restaurant,Sandwich Place,Bistro
4,Hotel Raffaello,Hotel,41.897770,12.495622,2,Italian Restaurant,Ice Cream Shop,Pizza Place,Bed & Breakfast,Wine Bar,Cocktail Bar,Café,Plaza,Sandwich Place,Park


In [47]:
k1=hotels_merged.loc[hotels_merged['Cluster Labels'] == 0, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k1)
display(hotels_merged.iloc[k1.index,0].tolist())
k1.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Hotel,Italian Restaurant,Café,Ice Cream Shop,Plaza,Bed & Breakfast,Cocktail Bar,Restaurant,Dessert Shop,Bookstore,Bistro
9,Roof Deck,Italian Restaurant,Café,Ice Cream Shop,Plaza,History Museum,Pizza Place,Bookstore,Bistro,Church,Bed & Breakfast
10,Hotel,Italian Restaurant,Café,Ice Cream Shop,Bed & Breakfast,Plaza,Pizza Place,Cocktail Bar,Bookstore,Bistro,Church
14,Hotel,Italian Restaurant,Ice Cream Shop,Café,Plaza,Bed & Breakfast,Cocktail Bar,Pizza Place,History Museum,Church,Fountain
16,Hotel,Italian Restaurant,Ice Cream Shop,Café,Plaza,Bed & Breakfast,Cocktail Bar,Bistro,Fountain,Dessert Shop,Cosmetics Shop
17,Hotel,Italian Restaurant,Café,Plaza,Ice Cream Shop,Bed & Breakfast,Bookstore,Bistro,Restaurant,Dessert Shop,Church
20,Hotel,Italian Restaurant,Pizza Place,Café,Bed & Breakfast,Ice Cream Shop,Plaza,Restaurant,Historic Site,Lounge,Cocktail Bar
23,Hotel,Italian Restaurant,Café,Ice Cream Shop,Pizza Place,Bed & Breakfast,Restaurant,Plaza,Lounge,Bakery,Irish Pub
25,Hotel,Italian Restaurant,Plaza,Park,Café,Roman Restaurant,Scenic Lookout,Ice Cream Shop,Gastropub,Fast Food Restaurant,Creperie
32,Hotel,Italian Restaurant,Ice Cream Shop,History Museum,Cocktail Bar,Café,Bed & Breakfast,Fountain,Movie Theater,Plaza,Restaurant


['Hotel Diana',
 'Hotel Mediterraneo',
 'iQ Hotel Roma',
 'Hotel Sonya',
 'Hotel Rex Rome',
 'Welcome Piram Hotel Rome',
 'Hotel Amalfi Rome',
 "Hotel D'Este",
 'Hotel San Anselmo',
 'Hotel Miami Rome',
 'Hotel Concorde',
 "Harry's Bar Trevi Hotel & Restaurant",
 'La Griffe Hotel Rome',
 'Hotel Orlanda',
 'Hotel Diocleziano',
 'Hotel Domus Romana',
 'Hotel Martini',
 'Hotel Alius']

(18, 11)

In [48]:
k2=hotels_merged.loc[hotels_merged['Cluster Labels'] == 1, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k2)
display(hotels_merged.iloc[k2.index,0].tolist())
k2.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hotel,Italian Restaurant,Historic Site,Temple,History Museum,Wine Bar,Plaza,Pizza Place,Monument / Landmark,Sandwich Place,Ice Cream Shop
1,Hotel,Italian Restaurant,Historic Site,Ice Cream Shop,History Museum,Plaza,Restaurant,Art Museum,Monument / Landmark,Pizza Place,Pub
8,Hotel,Italian Restaurant,Historic Site,Temple,Ice Cream Shop,Wine Bar,Sandwich Place,Pizza Place,Monument / Landmark,Plaza,Pub
28,Hotel,Italian Restaurant,Historic Site,Wine Bar,Pizza Place,Ice Cream Shop,Sandwich Place,Monument / Landmark,Temple,Café,Roman Restaurant
48,Hotel,Italian Restaurant,Historic Site,Plaza,Ice Cream Shop,Art Museum,History Museum,Restaurant,Museum,Pub,Monument / Landmark


['Hotel Forum',
 'Hotel Pace Helvezia',
 'Hotel Fori Imperiali Cavalieri',
 'Downtown Accommodation Hotel',
 'Hotel Cosmopolita Rome']

(5, 11)

In [51]:
k3=hotels_merged.loc[hotels_merged['Cluster Labels'] == 2, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k3)
display(hotels_merged.iloc[k3.index,0].tolist())
k3.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Hotel,Italian Restaurant,Pizza Place,Ice Cream Shop,Wine Bar,Coffee Shop,Bed & Breakfast,Sandwich Place,Historic Site,Indian Restaurant,Roman Restaurant
3,Hotel,Italian Restaurant,Ice Cream Shop,Bed & Breakfast,Pizza Place,Wine Bar,Plaza,Cocktail Bar,Roman Restaurant,Sandwich Place,Bistro
4,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Bed & Breakfast,Wine Bar,Cocktail Bar,Café,Plaza,Sandwich Place,Park
7,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Bed & Breakfast,Cocktail Bar,Plaza,Roman Restaurant,Art Museum,Sandwich Place
11,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Bed & Breakfast,Cocktail Bar,Plaza,Sandwich Place,Art Museum,Roman Restaurant
12,Hotel,Italian Restaurant,Pizza Place,Ice Cream Shop,Wine Bar,Bed & Breakfast,Café,Plaza,Cocktail Bar,Sandwich Place,Park
30,Hotel,Italian Restaurant,Pizza Place,Sandwich Place,Ice Cream Shop,Plaza,Café,Restaurant,Art Museum,Fountain,Trattoria/Osteria
38,Hotel,Italian Restaurant,Pizza Place,Ice Cream Shop,Plaza,Wine Bar,Café,Sandwich Place,Bed & Breakfast,Coffee Shop,Cocktail Bar
44,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Bed & Breakfast,Wine Bar,Café,Plaza,Cocktail Bar,Sandwich Place,Park
47,Hotel,Italian Restaurant,Pizza Place,Ice Cream Shop,Plaza,Wine Bar,Café,Coffee Shop,Sandwich Place,Bed & Breakfast,Cocktail Bar


['Hotel Apollo',
 'Hotel Galatea',
 'Hotel Raffaello',
 'Hotel Esposizione',
 'Hotel Giolli',
 'The Fifteen Keys Hotel',
 'Raphaël Hotel',
 'Hotel De Monti',
 'Hotel Saturnia',
 'B&B Hotel']

(10, 11)

In [52]:
k4=hotels_merged.loc[hotels_merged['Cluster Labels'] == 3, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k4)
display(hotels_merged.iloc[k4.index,0].tolist())
k4.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Hotel,Italian Restaurant,Boutique,Café,Plaza,Jewelry Store,Japanese Restaurant,Trattoria/Osteria,Pizza Place,Road,Historic Site
13,Hotel,Italian Restaurant,Trattoria/Osteria,Café,French Restaurant,Pizza Place,Plaza,Wine Bar,Church,Park,Beer Store
15,Hotel,Italian Restaurant,Restaurant,Trattoria/Osteria,Cocktail Bar,Pizza Place,Middle Eastern Restaurant,Plaza,Fountain,Bed & Breakfast,Roman Restaurant
18,Hotel,Italian Restaurant,Plaza,Ice Cream Shop,Pizza Place,Boutique,Trattoria/Osteria,Sandwich Place,Restaurant,Art Museum,Toy / Game Store
19,Hotel,Italian Restaurant,Bistro,Restaurant,Trattoria/Osteria,Pizza Place,Movie Theater,Cocktail Bar,Ice Cream Shop,Café,Plaza
21,Bed & Breakfast,Italian Restaurant,Ice Cream Shop,Boutique,Fountain,Plaza,Bed & Breakfast,Restaurant,Pizza Place,Café,Japanese Restaurant
22,Roof Deck,Italian Restaurant,Café,Historic Site,Trattoria/Osteria,Cocktail Bar,Plaza,French Restaurant,Wine Bar,Pizza Place,Gym / Fitness Center
24,Hotel,Italian Restaurant,Boutique,Plaza,Jewelry Store,Ice Cream Shop,Historic Site,Cosmetics Shop,Pizza Place,Café,Road
26,Hotel,Italian Restaurant,Bistro,Restaurant,Trattoria/Osteria,Cocktail Bar,Pizza Place,Movie Theater,Café,Gym,Nightclub
27,Hotel,Italian Restaurant,Boutique,Ice Cream Shop,Plaza,Café,Restaurant,Fountain,Bed & Breakfast,Japanese Restaurant,Trattoria/Osteria


['Hotel Hassler Roma',
 "Hotel Capo D'Africa Rome",
 'Hotel Ambasciatori Palace',
 '939 Hotel (Hotel Marcus)',
 'Grand Hotel Palace',
 'Hotel Sistina - Roma',
 'Hotel Gladiatori Palazzo Manfredi',
 'Hotel Art',
 'Baglioni Hotel Regina',
 'Hotel Cecil Rome',
 "Hotel Campo De' Fiori",
 'Hotel Concordia',
 'Edera Hotel',
 'Hotel Arenula',
 'Aleph Rome Hotel, Curio Collection by Hilton',
 'Hotel Scalinata di Spagna',
 'B&B Trevi Hotel']

(17, 11)

### Results and Discussion

Our analysis shows that exist 50 hotels registered by Foursquare in Rome in a circular area with a radius of 1.5km, also as we can expect the most common venue in all the Clusters is 'italian Restaurant', so that venue will not necessary in to the recommendation, also we cang see in the first map that the majority of hotels are in the city center, that could be a chracteristic to consider if our client prefer a quite place.Then we can analyze each Cluster and finally get a conclussion.

1) Cluster 1 contains 18 hotels with 10 tops venues mainly cotains Café/Ice cream shop/wine bar/pizza place...,it lookslike a place where the citizens will enjoy, but maybe not for a tourist who want try and taste differents things. 2) Cluster 2 contains 5 hotels with 10 tops venues mainly cotains Boutique/Ice cream shop/fountain/shops, it looks like for a tourist who go for shopping and have a good time. 3) Cluster 3 contains 10 hotels with 10 tops venues mainly cotains ice cream shop/plaza/historic sites..., it looks balanced for a tourist who want know about the history, but enjoy his free time too. 4) Cluster 4 contains 17 hotels with 10 tops venues mainly cotains history site/Temples/Museums/Monuments..., it unlike the first cluster looks good for a tourist who want know more about the culture and history of Rome.

### Conlusion

Purpose of this project was to identify a group of hotels for a tourists depending of their pleasures.Target to individuals who want to travel to Rome,Italy for sightseeing. The Foursquare location data was leveraged to compare each hotel to provide reliable suggestions for individuals who want to choose a hotel according their reasons to travel(sightseeing,business,shopping,etc).With unsupervised learning K-means algorithm, all hotels were clustered in to 4 categories, the advantages of each category was expressed to help individuals choose the ideal group oh hotels to recommend group of hotels to recommend to the client. This Project simply processed the hotels, and cluster them into 4 categories based one the venues around, the results can only help individuals choose the hotel they want. Further analysis can be done base on these 4 clusters, which can help provide more detail information to clarify the advantages of each.